<a href="https://colab.research.google.com/github/manuelpope/MscIA/blob/main/iaa2021_pac3_template_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### M0.539 · IAA · PEC3 · 2020-21 · 2
##### Màster universitari d'Enginyeria Computacional i Matemàtica
##### Universitat Oberta de Catalunya, Universitat Rovira i Virgili

# PEC3: OPTIMIZACIÓN

## INTRODUCCIÓN

En esta prueba de evaluación estudiaremos el comportamiento y características de los algoritmos genéticos mediante la implementación de una versión muy simple de ellos. 

## COMPETENCIAS

En este enunciado se trabajan las siguientes competencias generales de
máster:
- Capacidad para proyectar, calcular y diseñar productos, procesos e
instalaciones en todos los ámbitos de la ingeniería informática.
- Capacidad para el modelado matemático, cálculo y simulación en
centros tecnológicos y de ingeniería de empresa, particularmente en
tareas de investigación, desarrollo e innovación en todos los ámbitos
relacionados con la ingeniería informática.
- Capacidad para aplicar los conocimientos adquiridos y solucionar
problemas en entornos nuevos o poco conocidos dentro de contextos
más amplios y multidisciplinares, siendo capaces de integrar estos
conocimientos.
- Poseer habilidades para el aprendizaje continuo, autodirigido y
autónomo.
- Capacidad para modelar, diseñar, definir la arquitectura, implantar,
gestionar, operar, administrar y mantener aplicaciones, redes,
sistemas, servicios y contenidos informáticos.

Las competencias específicas de esta asignatura que se trabajan en esta
prueba son:
- Entender qué es el aprendizaje automático en el contexto de la
inteligencia artificial.
- Distinguir entre los diferentes tipos y métodos de aprendizaje.
- Aplicar las técnicas estudiadas a un caso real.

## RECURSOS

Esta PEC requiere los recursos siguientes:

Archivos proporcionados:

  * iaa2021_pac3_template_es.ipynb

Complementarios: 
  * Manual de teoría de la asignatura, **especialmente su apartado 5.5**.

## ENTREGA Y CRITERIOS DE EVALUACIÓN

La PEC se debe entregar el **25 de mayo de 2021**.

La entrega debe incluir una versión editada de este cuaderno (.ipynb). Se recomienda el uso de Google Colab (https://colab.research.google.com/). El código de las soluciones a los ejercicios se debe implementar y ejecutar en las celdas de código proporcionadas y la respuestas justificadas se deben agregar a las celdas de texto correspondientes.

Todas las respuestas deben estar correctamente razonadas y justificadas. **Las soluciones que no vayan acompañadas de la correspondiente respuesta razonada no serán evaluadas**.


## DESCRIPCIÓN DE LA PEC

En esta PEC vamos a implementar un algoritmo genético (GA) básico, analizaremos su comportamiento con algunas funciones a optimizar que poseen características interesantes para evaluar algoritmos de optimización, y razonaremos a nivel teórico las implicaciones de los distintos tipos de configuración del algoritmo.

Al final de la realización de esta PEC, el alumno debería tener la capacidad suficiente como para adaptar el algoritmo a distintos problemas de optimización, pero para la realización de ésta se proporcionan más adelante las codificaciones de tres de las funciones explicadas en:

> Anescu, George. "Scalable test functions for multidimensional continuous optimization". _UPB Scientific Bulletin, Series C: Electrical Engineering_ 79(4) (2017).

Se puede acceder a una versión completa del artículo [mediante este enlace de ResearchGate](https://www.researchgate.net/publication/321037474_Scalable_test_functions_for_multidimensional_continuous_optimization).




**MUY IMPORTANTE**

En los ejercicios en los que se solicita implementar código dentro de un fragmento de código dado, tan solo podéis incluir código entre los comentarios *# TO DO...* y *# END OF TO DO*:

```
  # TO DO: Do something...

  # Your code here ...
  
  # END OF TO DO
```

***No está permitido modificar código ya proporcionado ni escribir fuera de estas áreas (excepto si creáis funciones auxiliares propias que son llamadas desde dentro de las regiones indicadas). De lo contrario, el ejercicio será evaluado con un 0.***

<hr>

## EJERCICIO 1

Como se puede observar en el artículo anterior, nos basaremos en la optimización de funciones que admiten un número indeterminado de variables de entrada, todas dentro del dominio de los números reales.

Para la implementación de un algoritmo genético, un elemento esencial es la codificación de las posibles soluciones del problema, o dicho de otro modo, la codificación de los individuos que evolucionarán durante la ejecución del algoritmo a través de distintas poblaciones a las que se les aplicarán diferentes operadores evolutivos hasta alcanzar una condición de finalización, bien sea basada en el tiempo transcurrido, o en la calidad de los resultados obtenida.

En esta ocasión, la codificación que emplearemos será muy básica: si queremos optimizar las distintas funciones en un espacio N-dimensional, nuestros individuos consistirán simplemente en N valores reales. Cada uno de estos valores será uno de sus genes y corresponderá al valor de una variable de entrada de la función.

Así, si tenemos una función f<sub>1</sub>(**x**) donde **x**=(x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>, ..., x<sub>n</sub>), nuestro individuo consistirá en un array de *n* valores reales, y la función devuelve un valor a partir de esos *n* valores.

Un ejemplo gráfico para una función se muestra en el artículo referenciado. En el caso de f<sub>10</sub>, el artículo nos muestra que el mínimo teórico se encuentra en x = (0.4, 0.4) para n=2. El objetivo de nuestro algoritmo genético sería encontrar este valor mínimo.

Para facilitar ciertos cálculos durante la PEC, la representación de los individuos y demás operaciones estarán basados en [la representación de arrays de la librería numpy](https://numpy.org/doc/stable/reference/generated/numpy.array.html).

Así pues, nuestro primer paso será importar las librerías y útiles necesarios:



In [1]:
import numpy as np
from numpy.random import choice
import random

Teniendo en cuenta la definición de individuo que acabamos de realizar, una función sencilla que genera una población aleatoria de individuos de un determinado tamaño y sujetos a una serie de restricciones sería como sigue:

In [2]:
def generate_random_population(npop, limits=list(zip(np.zeros(5),np.ones(5))), ngenes=5):
  
  def new_ind():
    return [random.uniform(limits[i][0], limits[i][1]) for i in range(ngenes)]

  return np.array([new_ind() for n in range(npop)])



Como se puede observar, uno de los parámetros de entrada es una lista de tuplas que representan los límites mínimos y máximos de cada uno de los valores, definiendo así el dominio de la función. En este caso, el valor **por defecto** de este parámetro es una lista de 5 tuplas (0.0, 1.0), con lo que los valores mínimo y máximo serán 0 y 1, respectivamente. El valor del parámetro *ngenes* (longitud de cada individuo) ha de coincidir con la longitud de la lista *limits*.

**1.a) (0.5 PUNTOS) Explicad en qué consiste la función *generate_random_population*, qué significan sus argumentos y para qué se utilizan.**




**Respuesta:**

*En Terminos de Algortimos:*


*Las funciones generadoras regresan arreglos (vectores) estos basados en un calculo interno dentro la funcion, Para este caso sus entradas son el tamano de la poblacion , sus limites y la cantidad de diferentes genes que van a estar en el proceso.Teniendo como valores default catidad de genes igual a 5 y los limites una lista de 5 posiciones rellena de unos si no es tomado otro valor como argumento. En la iteracion de n habitantes se van generanto y retornando cada individuo_i este se realiza utilizando las funciones random con distribucion uniforme ,variando el valor de cada gen al azar entre 0 y 1.Hace uso de esta funcion recursiva y da una lista de listas con las probabilidades generadas en cada gen por cada individuo.*

**1.b) (0.5 PUNTOS) Para que la función *generate_random_population* funcione correctamente, es necesario realizar algunas comprobaciones. Redefinid la función de manera que se realicen las comprobaciones necesarias para que la población resultante tenga sentido y no haya errores de ejecución. Mostrad un mensaje de error en caso que no se cumplan las condiciones adecuadas.** 




In [3]:
def generate_random_population(npop, limits=list(zip(np.zeros(5),np.ones(5))), ngenes=5):
  # TO DO: Do something...
  try:
        test = npop/npop
        test = ngenes/ngenes
        if ngenes>len(limits) or not isinstance(ngenes,int):
            print("Its not a possible combination of ngenes>len(limits) or ngenes is not a Integer")
            return
    
  except (TypeError, ZeroDivisionError):
      raise print(" !!!!  npop or ngenes should not be zero !!!!")

  # END OF TO DO
  def new_ind():
    return [random.uniform(limits[i][0], limits[i][1]) for i in range(ngenes)]

  return np.array([new_ind() for n in range(npop)])

<hr>

Otro aspecto esencial para el funcionamiento de un GA es la puntuación que los diferentes individuos reciben como medio para evaluar cuán bueno es un individuo a la hora de optimizar nuestro problema. Así, cuando disponemos de una determinada población, hemos de ser capaces de evaluar todos los individuos y darles una puntuación, de manera que podamos ordenar la población según la bondad de sus individuos para resolver nuestro problema.

A continuación se definen 3 de las funciones discutidas en el artículo anteriormente referenciado, que serán la base de nuestros posteriores ejercicios. Éstas corresponden a las funciones f<sub>1</sub>, f<sub>8</sub> y f<sub>13</sub> del apartado 3 del artículo, donde podéis encontrar sus peculiaridades y valores óptimos teóricos. Las funciones reciben como argumento un individuo tal y como lo hemos definido anteriormente, y devuelven su correspondiente valor.

Por ejemplo, f<sub>1</sub> solo tiene un mínimo (unimodal) en x= (0,0,...,0) en el cual f<sub>1</sub> = 0.

Podéis definir libremente vuestras propias funciones alternativas para vuestras propias comprobaciones, aunque más adelante comprobaremos el funcionamiento del GA con estas 3 funciones. 

In [4]:
def fopt1(ind):
  
  x0 = [ind[len(ind)-1]]
  xlast = [ind[0]]
  working_array = np.concatenate((x0,ind,xlast))
  res = 0

  for j in range(1, len(ind)+1):
    res += (2*working_array[j-1] + (working_array[j]**2)*working_array[j+1] - working_array[j+1])**2
  
  return res

def fopt8(ind):
  
  x0 = [ind[len(ind)-1]]
  xlast = [ind[0]]
  working_array = np.concatenate((x0,ind,xlast))
  term = ((working_array[1]+1)**2)/len(ind)
  sum = 0

  for j in range(1, len(ind)+1):
    sum += (working_array[j] - 2*(working_array[j-1] + working_array[j+1]) - working_array[j-1]*working_array[j+1] - 2)**2
  
  return term + sum

def fopt13(ind):
  
  x0 = [ind[len(ind)-2]]
  xlast = [ind[0]]
  working_array = np.concatenate((x0,ind,xlast))
  sum0 = 0
  sum1 = 0

  for j in range(1, len(ind)+1):
    sum0 += (working_array[j-1]*working_array[j]**4 - working_array[j+1]*working_array[j]**3 + working_array[j-1]*working_array[j+1] - working_array[j+1])**2

  for j in range(len(ind)):
    sum1 += ind[j]

  return sum0 + sum1


<hr>

Estas funciones suponen un problema interesante a la hora de encontrar sus mínimos absolutos. Por ello, por el momento nos centraremos en el problema de minimizar una función. Consideraremos que nuestros problemas de optimización consisten en minimizar las funciones objetivo.

**1.c) (1 PUNTO) Implementar la función *eval_pop*, que recibe una determinada población de individuos y una función de evaluación (normalmente denominada *fitness function*) y devuelve una lista ordenada de tuplas, en las que el primer elemento es un individuo de la población y el segundo elemento es su correspondiente puntuación.
La lista de tuplas devuelta deberá estar ordenada en orden decreciente (el primer elemento es el que mejor resultado ha obtenido).** 




In [5]:
def eval_pop(pop, f):
  # Returns a list of tuples in descending order of the goodness of f. Shape of tuples are (individual, score), e.g., ([2.3,0.004,1,8.2,6], 0.361).
  
  # TO DO: Implement the function

  def takeSecond(elem):
    return elem[1]
  listMapFuntion = [f(elem) for elem in pop]
  sorted_list= list(zip(pop,listMapFuntion))
  sorted_list.sort(key=takeSecond,reverse=True)


  # END OF TO DO

  return sorted_list
  


<hr>

## EJERCICIO 2

Una vez tenemos nuestra población ordenada según la bondad del resultado de cada individuo, estamos en disposición de aplicar el resto de operadores genéticos. Un paso necesario para poder producir una siguiente generación de individuos es el de seleccionar las parejas que se cruzarán para tener descendencia.

Para ello, disponemos del siguiente código que, como se puede observar, tan solo trabaja con poblaciones de tamaño superior a 9:



In [6]:
def couples_selection(ordered_pop, n_elitism):

  if len(ordered_pop) < 10:
    print("Error: population's size should be higher than 9")
    return
  
  len_a = int(len(ordered_pop)/10)
  len_b = len_a * 3
  len_c = len_a * 4

  a = np.ones(len_a) * 0.5 / len_a
  b = np.ones(len_b) * 0.3 / len_b
  c = np.ones(len_c) * 0.15 / len_c
  d = np.ones(len(ordered_pop) - len_a*8)
  d = d * 0.05 / len(d)

  prob = np.concatenate((a,b,c,d))
  indices = range(len(ordered_pop))
  selected_indices = [choice(indices, 2, p=prob) for i in range(len(ordered_pop) - n_elitism)]
  couples = [[ordered_pop[i1], ordered_pop[i2]] for [i1,i2] in selected_indices]
  return np.array(couples)


**2.a) (1 PUNTO) Explicar detalladamente qué hace la función *couples_selection*: qué son los argumentos que recibe, qué devuelve, qué comprobaciones se hacen, cuál es el criterio de selección, el porqué de n_elitism, etc.**

**Respuesta:**

En general este metodo regresara n parejas para ser apareadas con el metodo que se desea posteriormente. El parametro ordered_pop es la poblacion ordenada
ya sea minimizada o maximicida por medio de una funcion fitness o objetivo.El parametro n_elitism es la cantidada de individuos en el top o los n que seran
pasados a la siguiente generacion sin ser modificados dado que ya son un individuo que el funcion objetivo postulo como los mejores de la generacion
Descripcion del metodo es el siguiente:
1. evalua que la pobalcion sea mayor a 9
2. genera un grupo de longitudes con el 10 , 30 y 40 del tamano de la poblacion
3. se generan vectores con probabilidades para almacenar los resultados de las descendencias
4. se pasa a hacer una iteracion de n tamano de poblacion menos n_elitismo factor de elitismo donde cada iteracion se selecciona al azar los dos indices de las parejas y su probabilidad, para luego contruir tantas tuplas con interaciones y retornar estas para el proceso de crossover.

<hr>


En esta versión simplificada de nuestro GA, consideraremos que cada pareja tan solo puede producir un descendiente, y que la manera de producirlo siempre será mediante la estrategia de cruce en dos puntos (**ver apartado 5.5.1 de los apuntes de la asignatura, en especial la figura 52**).

Como ya sabemos, en un paso evolutivo, después de haber seleccionado las diferentes parejas que tendrán descendencia, éstas se cruzan y, en este punto, pueden surgir mutaciones. 

En nuestra versión simplificada de nuestro GA, el resultado de todo este proceso sería el que devuelve la función *get_offspring* a continuación, que devuelve un array de individuos de igual tamaño que el array de parejas de entrada, y que tiene en cuenta los procesos de cruce y mutación anteriormente descritos.

**2.b) (2 PUNTOS) Teniendo en cuenta las funciones *mutate* y *get_offspring* proporcionadas a continuación, completar la definición de la función *crossover*, que recibe una pareja de individuos y devuelve un nuevo individuo resultado de aplicar la estrategia de cruce en 2 puntos. El tipo del resultado será *ndarray*. Podéis consultar la documentación de [numpy.concatenate](https://numpy.org/doc/stable/reference/generated/numpy.concatenate.html), por si os es útil para formar el nuevo individuo.** 

**NOTA: NO está permitido modificar las funciones *mutate* y *get_offspring*. La función *crossover* debe adaptarse a ellas teniendo en cuenta cómo es llamada. Es importante notar que la función *crossover* no necesita realizar ningún tipo de mutación, ya que se realiza después de su llamada.**

In [7]:
def mutate(ind, limits):
  # print("Mutating individual ", ind)
  factor = 1 + (0.2 * choice([-1,1], 1))
  gene_index = choice(range(len(ind)), 1)[0]
  mutated_val = ind.item(gene_index) * factor

  if mutated_val < limits[gene_index][0]:
    mutated_val = limits[gene_index][0]
  elif mutated_val > limits[gene_index][1]:
    mutated_val = limits[gene_index][1]

  ind[gene_index] = mutated_val

  return


def crossover(couple):
    ancestor1 = couple[0]
    ancestor2 = couple[1]
    # TO DO: Complete the crossover function according to what's described in
    # section 5.5.1 and figure 52 of the subject's materials.

    rangeX= range(0,len(ancestor1),4)



    offspring=np.zeros(len(ancestor1))

    for i in rangeX:
        if i+1 <= len(ancestor1)-1:
            offspring[i]=ancestor1[i]
            offspring[i+1]=ancestor1[i+1]
            ancestor2[i] = 0
            ancestor2[i+1] =0

    offspring=offspring+ancestor2

    return offspring



    # Your code here ...

    # END OF TO DO


  

def get_offspring(couples, mutp, limits):
  mp=mutp
  children = [crossover(couple) for couple in couples]
  mutation_roulette = [choice([True, False], 1, p=[mp, 1-mp]) for _ in children]
  children_roulette = list(zip(children, mutation_roulette))

  for child in children_roulette:
    if child[1][0]:
      mutate(child[0], limits) 
      # print("Mutated: ",child[0])

  return np.array([child[0] for child in children_roulette])
    
  



In [8]:
# El codigo de Crossover lo que realiza en pasos es:
# 1. separa los dos ancestros de la tupla couple
# 2. se realiza un rango de 0 a n con saltos de 4 por que cada
#  secuencia de dos genes heredara el progenitor de diferente ancestro
# 3. se crea un vector de ceros llamado offspring de tamano de los ancestros
# 4. se realizan los cambios en el ancestro 2 y el offspring de tal manera que
# Donde hereda del ancestro uno el ancestro dos se llenara con zeros para al final 
# y offspring tomara valores en esa posicion del ancestro 1
# retornara el vector suma de offspring + ancestro dos y completara el crossover de
# dos puntos.

<hr>

## EJERCICIO 3

En este punto, ya disponemos de los operadores de generación de población aleatoria, evaluación, selección, cruce y mutación, así como de una función que nos genera la descendencia de una serie de parejas. ¡Parece que estamos listos para completar nuestro algoritmo genético!  



**3) (2.5 PUNTOS) Completad la siguiente definición de *runGA*, que recibe un tamaño de población (*npop*), un tamaño de individuo (*ngenes*), unos límites para cada gen de los individuos (*limits*), una función objetivo (*fitness*) para evaluar a cada individuo, un factor de elitismo (*nelitism*), una probabilidad de mutación (*mutp*) y un número de generaciones (*ngenerations*).**

La función aplicará los distintos operadores evolutivos EN EL ORDEN ADECUADO durante *ngenerations*, y dará como resultado el individuo ganador de la población correspondiente a la última generación junto con su puntuación obtenida.

**MUY IMPORTANTE: Explicad y justificad vuestra implementación. La entrega de código sin razonamiento ni justificación será evaluada con un 0, aunque el algoritmo funcione correctamente.** 

NOTA 1: Podéis definir todas las funciones auxiliares que consideréis oportunas, o realizar todos vuestros cálculos dentro de la propia función *runGA*. En cualquier caso, es necesario que expliquéis y justifiquéis debidamente todas las acciones que lleváis a cabo.

NOTA 2: Una posible ejecución de este algoritmo podría ser la siguiente:

````
> runGA(1000, 5, list(zip(np.ones(5)*-2,np.ones(5)*2)), fopt13, 4, 0.4, 25)

Winner after generation 0 : (array([-1.16228187, -0.92100491, -0.71327896,  0.20645224, -0.31941844]), -1.3719550258326878)
Winner after generation 1 : (array([-1.16228187, -0.92100491, -0.71327896,  0.20645224, -0.31941844]), -1.3719550258326878)
Winner after generation 2 : (array([-0.98088075, -0.56069746, -0.74013361, -0.94272366, -1.05304763]), -1.504570220226677)
Winner after generation 3 : (array([-0.98088075, -0.81577616, -0.74013361, -0.94272366, -1.05304763]), -2.6628565757698066)
Winner after generation 4 : (array([-0.98088075, -0.86245533, -0.99179012, -0.94272366, -1.05304763]), -3.8677707421578926)
Winner after generation 5 : (array([-0.98088075, -0.86245533, -0.99179012, -0.94272366, -1.05304763]), -3.8677707421578926)
Winner after generation 6 : (array([-0.98088075, -0.86245533, -0.99179012, -0.94272366, -1.05304763]), -3.8677707421578926)
Winner after generation 7 : (array([-0.98088075, -1.03494639, -0.99179012, -0.94272366, -1.05304763]), -4.56395601696479)

...

Winner after generation 24 : (array([-0.98088075, -0.99354853, -0.99179012, -0.94272366, -1.01092573]), -4.748795961685159)

Absolute winner:
(array([-0.98088075, -0.99354853, -0.99179012, -0.94272366, -1.01092573]),
 -4.748795961685159)
````


In [9]:
def runGA(npop, ngenes, limits, fitness, nelitism, mutp, ngenerations):
  pop_0 = generate_random_population(npop, limits, ngenes)
  sorted_pop_with_score = eval_pop(pop_0, fitness)
  new_pop = np.array([p[0] for p in sorted_pop_with_score])
  
  

    # TO DO: Complete your GA!

  generation=[]
  for g in range(ngenerations):
    lg = couples_selection(new_pop,nelitism)
    generation=get_offspring(lg,mutp,limits)
    winner= eval_pop(generation, fitness)
    new_pop = np.array([p[0] for p in winner])
    if sorted_pop_with_score[0][1]<winner[0][1]:
      print(g)
      sorted_pop_with_score=winner

    


  
    # END OF TO DO
    
    print("Winner after generation", g, ":",winner[0])

  print("Absolute winner:")
  return sorted_pop_with_score[0]

In [10]:
runGA(1000, 5, list(zip(np.ones(5)*-2,np.ones(5)*2)), fopt1, 4, 0.4, 20)

0
Winner after generation 0 : (array([1.8680189 , 1.75522559, 1.67055536, 1.82693336, 1.9229182 ]), 302.4818449974381)
1
Winner after generation 1 : (array([-2.        , -1.99773222, -1.63953305, -1.98494341, -1.76820818]), 363.5439878935335)
2
Winner after generation 2 : (array([1.83037475, 1.97557336, 1.88913834, 2.        , 1.82967806]), 389.89073502200137)
3
Winner after generation 3 : (array([-2.        , -2.        , -1.96743966, -1.98494341, -1.76820818]), 434.0954314339037)
4
Winner after generation 4 : (array([2.        , 1.97557336, 2.        , 2.        , 1.9229182 ]), 474.3803974490053)
5
Winner after generation 5 : (array([2.        , 1.97557336, 2.        , 2.        , 2.        ]), 493.71841737241886)
6
Winner after generation 6 : (array([2., 2., 2., 2., 2.]), 500.0)
Winner after generation 7 : (array([2., 2., 2., 2., 2.]), 500.0)
Winner after generation 8 : (array([2., 2., 2., 2., 2.]), 500.0)
Winner after generation 9 : (array([2., 2., 2., 2., 2.]), 500.0)
Winner after

(array([2., 2., 2., 2., 2.]), 500.0)

**RAZONAMIENTO Y JUSTIFICACIÓN:**


Para realizar este metodo de runGA donde recibe por paramtros muchos argumentos
debido que este seria el main donde llama y utiliza todos los metodos definidos previamente. 
1. npop -> tamano de la poblacion que se creara en primera instancia
2. ngenes -> cantidad de genes o features que tendra cada individuo
3. limits -> los limites en la generacion de esta poblacion
4. fitness -> funcion objetivo donde se evaluaran los individuos mejor dotados
5. nelitism -> cuantos individuos pasaran a la siguiente ronda sin ser    modificados
6. mutp -> probabilidad que sucedan mutaciones 
7. ngenerations -> numero de iteraciones o generaciones que se correra el el metodo.

En terminos generales es un metodo iterativo que en cada corrida genera nuevas
parejas para hacer una generacion_i nueva tomando el factor de elitismo, con estas parejas va generar efectivamente los nuevos individuos con sus limites , probabilidad de mutacion y parejas. Se correra el algoritmo de ordenamiento sobre esta generacion_i obteniendo los mejores bajo la funcion objetivo, e imprimiendo en consola si el mejor individuo de la generacion presente es mejor que el mejor individuo en la historia de generaciones , es decir imprimiento el
mejor resultado posible y en que generacion sucedio, como ultimo se retorna  el mejor individuo de todas las generaciones calculadas. 

<hr>

## EJERCICIO 4

Ya disponemos de un GA implementado. Ahora vamos a ponerlo a prueba y razonar sobre su comportamiento y características.


**4.a) (1 PUNTO) Realizad varias ejecuciones del algoritmo para tratar de optimizar las funciones *fopt1, fopt8 y fopt13* proporcionadas en el ejercicio 1. Utilizad diferentes combinaciones de parámetros que afecten al proceso evolutivo y explicad la influencia de cada uno de ellos según los valores que les asignáis.**

**Considerad la descripción que se da de cada una de las funciones en el artículo referenciado. Esto os ayudará a evaluar el rendimiento de vuestro algoritmo en función del valor de sus parámetros. Fijáos en los valores mínimos teóricos que se indican.**

**NOTA: Podéis fijar *ngenes* con valor 5 para vuestros análisis.**

**NOTA 2: Prestad atención a los dominios de las funciones indicados en el artículo a la hora de fijar los límites: para *fopt1* y *fopt13* los límites son -2 y 2, mientras que para *fopt8* los límites son -3 y 3.**

**Respuesta:**



*fopt1[-2,2] -> Se observa como se llegan a que todos los genes tiene un valor de 2 o -2 en 7(avg) generaciones y puntuacion de 500 .
 fopt8 [-3.3]-> Se observa como se llegan a que todos los genes tiene un valor de 3 en solo 6(avg) generaciones y puntuacion de 2003.2
 fopt13[-2,2]-> Se observa como se llegan a que todos los genes tiene un valor de 2 o -2 en 4(avg) generaciones y puntuacion de 10134*

ESTAS DIFERENCIAS OCURREN POR QUE ALGUNAS FUNCIONES SON DE MAYOR COSTE COMPUTACIONAL, O EL METODO DE EVALUACION DE INDIVIDUOS ES DECIR LA FUNCION OBJETIVO RESULTA MAS OPTIMA BAJO ESOS PARAMETROS, POR ESO VEMOS DIFERENTES
CANTIDADES EN CUANTAS GENERACIONES SON NECESARIAS PARA ALCANZAR EL MEJOR VALOR DE GENES, YA SE UNA MAXIMIZACION O MINIMIZACION.




**4.b) (0.5 PUNTOS) ¿Cuál es la condición de finalización de nuestro GA en su estado actual? Explicad una alternativa razonable para finalizar la ejecución el algoritmo, justificando los motivos para implementarla, sus características, así como precauciones, ventajas y desventajas que pueda presentar.**


**NOTA: No hace falta implementar ningún código, pero sí justificar adecuadamente la respuesta.**

**Respuesta:**
*Deberiamos fijar los early stoppers para no hacer computo innecesario, de este modo podriamos fijar una tolerancia en la varianza de la funcion objetivo de tal manera que cuando no cambie en la iteracion sigueinte por mas de un 1% sencillamente hacer un break del ciclo y retornar el resultado, ex:

      diff = winner[0][1] - sorted_pop_with_score[0][1]
      sorted_pop_with_score=winner
      if diff < (winner[0][1]/100):
        break

```
# 
```


*

**4.c) (0.5 PUNTOS) ¿Qué modificaciones haría falta realizar para que nuestro GA aceptase tanto problemas de maximización como de minimización?**

**NOTA: No hace falta implementar ningún código, pero sí justificar adecuadamente la respuesta.**

**Respuesta:**
*Para este Caso deberia aceptar un booleano maxim=True/False,
Donde pasaria por un ciclo if y en vez de retornar los primeros de
cada generacion retornaria los ultimo de cada una, maxim default = False.
Asu vez la funcion eval_pop tendria un extra parametro con ese maxim para determinar su bloque del sort.*

def runGA(npop, ngenes, limits, fitness, nelitism, mutp, ngenerations,maxim=False):


```
# def runGA(npop, ngenes, limits, fitness, nelitism, mutp, ngenerations,maxim=False):

def eval_pop(pop_0, fitness, maxim):
```



**4.d) (0.5 PUNTOS) Se nos plantea que modifiquemos nuestro GA para que sea multiobjetivo. Es decir, que tenga en cuenta 2 funciones *fitness* a la hora de evaluar una solución, pudiendo requerir minimizar ambas, maximizar ambas, o minimizar una y maximizar la otra. ¿Qué modificaciones deberíamos implementar para conseguirlo?**

**NOTA: No hace falta implementar ningún código, pero sí justificar adecuadamente la respuesta.**

**Respuesta:**
*En primer lugar agregar si es max o min cada una de las funciones,aparte de enviar el array de funciones,Posterioremente en cada iteracion evaluar la funcion_i y se deberia definir una convencion que entre indice del array menor peso para asi dar herarquia al ordenamiento, algo similar a lo que sucede cuando ordenamos alfabeticamente una palabra, es decir tomar un top y ordenar por funcion objetivo 1 , si hay valores iguales el segundo criterio la funcion objetivo 2 y asi sucesivamente*